<h1><center>第3章 分组</center></h1>

In [2]:
import pandas as pd
import numpy as np

### 一、SAC过程
#### 1. 所谓SAC过程，指的是分组操作中的split-apply-combine过程，其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构。
#### 2. 在apply过程中，我们实际往往会遇到四类问题：整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数），变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化），过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组），综合问题——即前面提及的三种问题的混合。

### 二、groupby函数
#### 1. 分组函数的基本内容：

In [3]:
# groupby对象只是一个容器，它本身不会返回任何东西，只有当相应的方法被调用时才会返回对应结果
# [表1] 第一列为名字，有两个人，第二列为第几天，一共四天，第三列到第五列为不同类型的数据
df1 = pd.read_csv('data/ch3.csv')
# 1.1 根据某一列分组
group_single = df1.groupby('Name')
group_single.get_group('Bob')
# 1.2 根据某几列分组
group_mul = df1.groupby(['Name','Day_num'])
group_mul.get_group(('Bob','day1'))
# 1.3 组的大小、数量及其遍历
group_single.size() #组的大小
group_single.ngroups
for name,group in group_single:
    #print(name)
    #print(group)
    pass
group_mul.size()
group_mul.ngroups
for name,group in group_mul:
    #print(name)
    #print(group)
    pass
# 1.4 groupby函数中的level参数
# level参数针对的是MultiIndex型
# df1.groupby('Name',level=0).sum() # 此时指定level为默认索引
# df1.groupby('Name',level=1).sum() # 报错，因为只有单层索引

df1_mul_index = df1.set_index(['Name','Day_num'])
df1_mul_index.groupby(level=1).sum()
df1_mul_index.groupby(level=[0,1]).sum() # level=['Name','Day_num']结果也是一样的
df1_mul_index.groupby(['Name','Day_num']).sum().head() # 事实上直接使用索引进行传入也是一样的

Score1  Score2
Name Day_num                
Bob  day1         10    1.17
     day2          8    0.60
     day3         14    0.61
     day4          6    1.13
     day5          5    1.50

#### 2. 分组对象的一些重要说明：

In [5]:
# 事实上不一定要传入某一列作为分组对象，只要索引长度与df分割方向的长度相同即可，下面举两个例子：
# (a) 按照某个函数规则分组（比如奇数索引和偶数索引）
df1.groupby(lambda x:x%2==0).groups
'''
如果运行下面的程序，我们可以猜想，传入函数的机制就是传入了索引，因此如果传入了'col'列，那么就相当于对该列按索引取元素分组
def f(x):
    print(x)
    print(type(x))
df1.groupby(f).groups
_df1 = pd.read_csv('data/ch3.csv',index_col='Name')
def f(x):
    print(x)
    print(type(x))
_df1.groupby(f).groups
# _df1.groupby(lambda x:x%2==0).groups # 这一行报错，因为索引不是数值类型
'''
# (b) 多层索引
_df1 = pd.read_csv('data/ch3.csv',index_col=['Name','Score3'])
# 对所有第五列分数为A的分组，计算两人第四列分数的总和
_df1.groupby(lambda x:'%s'%x[0]+' %s'%('A' if x[1][0]=='A' else 'not A'))['Score2'].sum()

Bob A        1.52
Bob not A    5.16
Tom A        4.18
Tom not A    3.43
Name: Score2, dtype: float64

#### 3. 分组对象的所有属性或方法

In [4]:
print([attr for attr in dir(group_single) if not attr.startswith('_')])

['Day_num', 'Name', 'Score1', 'Score2', 'Score3', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


### 三、整合操作（Aggregation）
#### 1. 常见的整合函数（这些函数的特点都是返回单个标量值）：<br>
    mean()	    组内均值<br>
    sum()	    组内综合<br>
    size()	    组容量<br>
    count()	    组非缺失值个数<br>
    std()	    组标准差<br>
    var()	    组方差<br>
    sem()	    组内标准误差（$\frac{组内标准差}{\sqrt{组内非缺失值个数}}$）<br>
    describe()	组内统计信息汇总<br>
    first()	    组内第一个元素<br>
    last()	    组内最后一个元素<br>
    nth()	    组内第n个元素<br>
    min()	    组内最小值<br>
    max()       组内最大值

In [5]:
grouped = df1[['Score1','Score2','Name']].groupby('Name')
grouped.max()
# 调用reset_index或设置as_index=False可以返回不带行索引的整合数据
df1[['Score1','Score2','Name']].groupby('Name').max().reset_index()
df1[['Score1','Score2','Name']].groupby('Name',as_index=False).max()

,Name,Score1,Score2
0,Bob,8,0.86
1,Tom,5,0.88


In [6]:
# 验证标准误差的计算公式：
grouped.std().values/np.sqrt(grouped.count().values) == grouped.sem().values

array([[ True,  True],
       [ True,  True]])

#### 2. 同时使用多个整合函数：

In [7]:
agged = grouped.agg(['sum','mean','std'])
grouped.agg([('rename_sum','sum'),('rename_mean','mean')]) # 可以用元组重命名
grouped.agg({'Score1':['mean','max'],'Score2':'median'}) # 指定哪些函数作用哪些列
# 由于返回的还是DataFrame，我们尝试按照三个统计指标，在列（axis参数设置为1，默认为0）的第二层索引上进行分割：
agged.groupby(axis=1,level=1).mean().T

Name,Bob,Tom
mean,2.488571,2.414643
std,0.977015,0.547055
sum,34.840000,33.805000


#### 3. 使用自定义函数：

In [8]:
grouped = df1[['Score1','Score2','Name','Day_num']].groupby(['Name','Day_num'])
# 定义极差函数：
def R(df):
    # 这里的df相当于分组对象中的某一个组
    return df.max()-df.min()
grouped.agg(R)
# 使用lambda表达式也是合法的：
grouped.agg(lambda x:x.max()-x.min())
# 对新指标列取新名字(此处不难发现可以对同一个列进行多次聚合，但是不能使用lambda函数，会报错)：
grouped.agg(min_score1=pd.NamedAgg(column='Score1', aggfunc=R),
            max_score1=pd.NamedAgg(column='Score1', aggfunc='max'),
            range_score2=pd.NamedAgg(column='Score2', aggfunc=R)).head()

min_score1  max_score1  range_score2
Name Day_num                                      
Bob  day1              0           5          0.11
     day2              0           4          0.10
     day3              2           8          0.13
     day4              0           3          0.43
     day5              1           3          0.22

### 四、变换操作（Transformation）
#### 1. transform函数针对的是组内的每一个元素，每组返回的长度需要与组大小一致，分组标准化就是一种非常常见的变换操作：

In [9]:
index = np.random.randint(0,4,300)
index2 = np.random.randint(0,4,300)
column = np.random.rand((300))

In [10]:
df2 = pd.DataFrame({'col1':column,'col2':index})
df2.groupby([index2,index]).sum().head()

col1  col2
0 0   7.293841     0
  1  10.937373    24
  2  11.143768    56
  3   6.445654    42
1 0  11.660789     0

In [11]:
df2 = pd.DataFrame({'col':column},index=index)
transformed = df2.groupby(index).transform(lambda x:(x-x.mean())/x.std())
df2.groupby(index).mean()
df2.groupby(index).var()
transformed.groupby(index).mean()
transformed.groupby(index).var()

,col
0,1.0
1,1.0
2,1.0
3,1.0


In [12]:
# 如果返回的是标量，那么transform方法的每个组都会被广播为这个标量值：
df2.groupby(index).transform('max').sort_index()
# 同样也可以使用lambda函数和自定义函数符号：
df2.groupby(index).transform(lambda x:x.max()-x.min()).sort_index()
df2.groupby(index).transform(R).sort_index().head()

,col
0,0.967134
0,0.967134
0,0.967134
0,0.967134
0,0.967134


#### 2. 除了标准化外，另一个常见操作是对于缺失值的组内均值填充：

In [13]:
index = np.random.randint(0,4,300)
column = np.random.rand((300))
column[np.random.randint(0,300,20)] = [np.nan]*20 # 随机挑选20个位置设为缺失值
df2 = pd.DataFrame({'col':column},index=index)

In [14]:
df2['col'].hasnans # 判断是否有缺失值
transformed = df2.groupby(index).transform(lambda x: x.fillna(x.mean()))
transformed['col'].hasnans # 判断是否没有缺失值
mean1 = df2.groupby(index).mean()
mean2 = transformed.groupby(index).mean()
mean1-mean2<1e-15 # 判断均值是否相同（均值填充缺失值不影响均值）

,col
0,True
1,True
2,True
3,True


## 五、过滤操作（Filteration）
#### 1. 通过过滤操作能够筛选具备某些条件的组，比如选出Score2最大值不超过0.8的组：

In [15]:
df3 = pd.read_csv('data/ch3.csv')
# df3.groupby('Day_num').filter(lambda x:print(x))
# 从上面的打印内容可以看出，filter内传入的都是根据分层的数据子框，因此我们可以使用框的操作
df3.groupby('Day_num').filter(lambda x:x['Score2'].max()>=0.8).groupby('Day_num').max()
# filter返回的还是DataFrame，因此可以继续调用groupby

,Name,Score1,Score2,Score3
Day_num,,,,
day2,Tom,5,0.88,B-
day5,Tom,5,0.88,B-
day7,Tom,7,0.88,B-


### 六、apply函数

#### 1. 灵活性：

In [16]:
#df3.groupby('Day_num').apply(lambda x:print(x))
# 从上面打印内容可以看出，apply传入的内容同样是数据库但不同之处在于下面：
#df3.groupby('Day_num').filter(lambda x:x.describe) # 这条语句报错，因为filter只会接受返回布尔值
df3.groupby('Day_num').apply(lambda x:x.describe())
# 因此，apply是更为灵活的函数，它的返回值也非常灵活
# (a)每个组返回标量
df3.groupby('Day_num').apply(lambda x:x['Score1'].max()+x['Score2'].min())
# (b)每个组返回序列
df3.groupby('Day_num').apply(lambda x:x['Score1']-x['Score2'])
# (c)每个组返回数据框
df3 = pd.read_csv('data/ch3.csv')
df_applied = df3.groupby('Day_num').apply(lambda x:pd.DataFrame({'col1':x['Score1']-x['Score1'].mean()
                                                                ,'col2':x['Score2']-x['Score2'].mean()}))
df_applied.index = df3['Day_num']
df_applied.head()

,col1,col2
Day_num,,
day1,1.0,0.0900
day1,-1.0,0.2300
day2,-0.5,-0.1825
day2,0.5,0.4475
day3,2.0,-0.0750


#### 2. 同时统计多个指标：

In [17]:
from collections import OrderedDict
def f(df):
    data = OrderedDict()
    data['sum'] = df['Score1'].sum()
    data['var'] = df['Score1'].var()
    data['mean'] = df['Score1'].mean()
    return pd.Series(data)
group_single.apply(f)

,sum,var,mean
Name,,,
Bob,63.0,3.038462,4.500000
Tom,60.0,0.681319,4.285714


### 七、常用技巧
#### 1. 查看每一组的第一行或最后一行：

In [18]:
group_mul.head(1)
group_mul.tail(1)
group_mul.tail(1).head() # 对最后一行的表而言，查看该表的前几行
group_mul.first() # 区别在于是否分组
group_mul.last().head()

Score1  Score2 Score3
Name Day_num                       
Bob  day1          5    0.53      B
     day2          4    0.35     B-
     day3          6    0.24     C+
     day4          3    0.78     C+
     day5          3    0.86      B

#### 2. 带参数的整合函数：

In [19]:
#判断是否组内'Score1'所有的值都在5-6之间：
def f(s,low,high):
    return s.between(low,high).min()
group_mul['Score1'].agg(f,5,6)
#如果要呈现多个整合函数结果，那么参数无法直接传递，此时要使用wrapper方法：
def agg_f(f,name,*args,**kwargs):
    def wrapper(x):
        return f(x,*args,**kwargs)
    wrapper.__name__ = name
    return wrapper
new_f = agg_f(f,'if_in_5_6',5,6)
group_mul['Score1'].agg([new_f,'mean']).head()

if_in_5_6  mean
Name Day_num                 
Bob  day1          True   5.0
     day2         False   4.0
     day3         False   7.0
     day4         False   3.0
     day5         False   2.5

#### 3. 连续变量分组：

In [20]:
bins = [-np.inf,0.3,0.5,0.8,np.inf ]
cuts = pd.cut(df1['Score2'],bins=bins) #可选label添加自定义标签
df1.groupby(cuts)['Score1'].count()

Score2
(-inf, 0.3]     9
(0.3, 0.5]      4
(0.5, 0.8]     11
(0.8, inf]      4
Name: Score1, dtype: int64

### 八、问题与练习

#### 1. rolling和expanding方法从原理上说都是一种transform方法，请问它们有什么区别？

#### 2. 什么是fillna的前向/后向填充，如何实现？

#### 3. 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。

In [21]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)

#### 4. 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。

#### 5. idxmax和nunique是什么函数，它具有哪些功能和应用？

#### 【综合练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：
#### (a) 在所有重量超过1克拉的钻石中，价格的极差是多少？
#### (b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的递增价格序列长度的最大值。（参考问题3）
#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

#### 【综合练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：
#### (a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？
#### (b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。
#### (c) 图是建立模型的重要方法，将所有州的所有县视作图的顶点，那么整个图就能用一张横纵索引为县的表记录现在以如下规则建立图的边：从10年到17年，如果存在3种药物报告数在县i的减少数量超过该药全部报告数的20%，同时这3种药在县j增加了该药全部报告数的60%，就在第i行第j列的数据框中记做1，否则记做0，请制作一张符合如上要求的DataFrame并以csv格式保存在当前文件夹。